In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/forest-cover-type-prediction/train.csv.zip
/kaggle/input/forest-cover-type-prediction/sampleSubmission.csv
/kaggle/input/forest-cover-type-prediction/sampleSubmission.csv.zip
/kaggle/input/forest-cover-type-prediction/test3.csv
/kaggle/input/forest-cover-type-prediction/train.csv
/kaggle/input/forest-cover-type-prediction/test.csv
/kaggle/input/forest-cover-type-prediction/test.csv.zip


In [2]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, log_loss
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

rseed = 71

In [3]:
# ----------------------------------------------
# データ読み込み
# ----------------------------------------------
DATA_DIR = "/kaggle/input/forest-cover-type-prediction"
train = pd.read_csv(f"{DATA_DIR}/train.csv")
test  = pd.read_csv(f"{DATA_DIR}/test.csv")

# データ確認用
# print(train)
# print(test)

# 元テーブルからID列と目的変数をコピー
train_id = train["Id"].copy()
test_id  = test["Id"].copy()
train_y  = train["Cover_Type"].copy()

In [4]:
# ----------------------------------------------
# 特徴量作成(ベースライン)
# ----------------------------------------------

# 元テーブルから特徴量をコピー（目的変数は削除）
# train_x = train.copy().drop(columns=["Cover_Type", "Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways", "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", "Horizontal_Distance_To_Fire_Points", "Wilderness_Area1", "Wilderness_Area2", "Wilderness_Area3", "Wilderness_Area4", "Soil_Type1", "Soil_Type2", "Soil_Type3", "Soil_Type4", "Soil_Type5", "Soil_Type6", "Soil_Type7", "Soil_Type8", "Soil_Type9", "Soil_Type10", "Soil_Type11", "Soil_Type12", "Soil_Type13", "Soil_Type14", "Soil_Type15", "Soil_Type16", "Soil_Type17", "Soil_Type18", "Soil_Type19", "Soil_Type20", "Soil_Type21", "Soil_Type22", "Soil_Type23", "Soil_Type24", "Soil_Type25", "Soil_Type26", "Soil_Type27", "Soil_Type28", "Soil_Type29", "Soil_Type30", "Soil_Type31", "Soil_Type32", "Soil_Type33", "Soil_Type34", "Soil_Type35", "Soil_Type36", "Soil_Type37", "Soil_Type38", "Soil_Type39", "Soil_Type40"])
# test_x  = test.copy().drop(columns=["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways", "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", "Horizontal_Distance_To_Fire_Points", "Wilderness_Area1", "Wilderness_Area2", "Wilderness_Area3", "Wilderness_Area4", "Soil_Type1", "Soil_Type2", "Soil_Type3", "Soil_Type4", "Soil_Type5", "Soil_Type6", "Soil_Type7", "Soil_Type8", "Soil_Type9", "Soil_Type10", "Soil_Type11", "Soil_Type12", "Soil_Type13", "Soil_Type14", "Soil_Type15", "Soil_Type16", "Soil_Type17", "Soil_Type18", "Soil_Type19", "Soil_Type20", "Soil_Type21", "Soil_Type22", "Soil_Type23", "Soil_Type24", "Soil_Type25", "Soil_Type26", "Soil_Type27", "Soil_Type28", "Soil_Type29", "Soil_Type30", "Soil_Type31", "Soil_Type32", "Soil_Type33", "Soil_Type34", "Soil_Type35", "Soil_Type36", "Soil_Type37", "Soil_Type38", "Soil_Type39", "Soil_Type40"])
train_x = train.copy().drop(columns=["Cover_Type"])
test_x = test.copy()

In [5]:
# ID列を削除
train_x = train_x.drop(columns=["Id"])
test_x  = test_x.drop(columns=["Id"])

In [6]:
# ----------------------------------------------
# 使用する特徴量の可視化
# ----------------------------------------------

# 各特徴の統計量
# print(pd.concat([train_x, train_y], axis=1).describe())

# 各特徴のヒストグラム
# train_x.hist(bins=100, color="blue", grid=True, label='pandas')
# plt.show()

# for column in train_x.columns:
#     plt.figure(figsize=(10, 6)) # グラフのサイズを調整して見やすくする
#     sns.histplot(data=train_x, x=column, bins=100, color="blue", kde=True)
#     # kde=True を追加すると、ヒストグラムの上にカーネル密度推定の線が重なり、分布の形状がより分かりやすくなります。
#     plt.title(f'Distribution of {column}') # 各グラフに特徴量名をタイトルとして追加
#     plt.xlabel(column) # X軸ラベルも特徴量名にする
#     plt.ylabel('Count') # Y軸ラベル
#     plt.grid(True) # グリッド線を表示 (元のコードに合わせる)
#     plt.show()

In [7]:
# ------------------------------
# XGBoostの学習・推論・submit（ベースライン）
# ------------------------------
# 学習用にクラスラベルを修正（０〜６に対応させる）
train_y = train_y - 1

model = XGBClassifier(objective="multi:softmax", num_class=7, random_state=rseed)
model.fit(train_x, train_y)
preds = model.predict(test_x)

print("\n--- 学習済みモデルの性能評価 ---")
train_preds = model.predict(train_x)
train_preds_proba = model.predict_proba(train_x)
accuracy = accuracy_score(train_y, train_preds)
logloss = log_loss(train_y, train_preds_proba)
print(f"訓練データに対する正解率 (Accuracy): {accuracy:.4f}")
print(f"訓練データに対するLogLoss: {logloss:.4f}")


# 提出用にクラスラベルを修正（１〜７に対応させる）
preds = preds + 1

submission = pd.DataFrame({
    "Id": test_id,
    "Cover_Type":  preds
})

submission.to_csv("submission.csv", index=False)


--- 学習済みモデルの性能評価 ---
訓練データに対する正解率 (Accuracy): 0.9838
訓練データに対するLogLoss: 0.1183


In [8]:
# EDA用にデータフレームのコピーを作成（元のデータフレームを汚さないため）
df_eda = train.copy()

# =============================================================================
# 特徴量の定義と前処理 (EDA用)
# =============================================================================
# 特徴量のグループ分け
numerical_features = [
    'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points'
]
wilderness_features = [col for col in df_eda.columns if 'Wilderness_Area' in col]
soil_features = [col for col in df_eda.columns if 'Soil_Type' in col]

# One-Hotエンコードされたカテゴリカル特徴量を元の単一カラムに戻す
# これにより、グループごとの集計や可視化が容易になります。
df_eda['Wilderness_Area'] = df_eda[wilderness_features].idxmax(axis=1)
df_eda['Soil_Type'] = df_eda[soil_features].idxmax(axis=1)

In [9]:
# =============================================================================
# 1. 目的変数（Cover_Type）の分布を確認
# =============================================================================
print("\n--- 1. 目的変数（Cover_Type）の分布 ---")
# ここでは df_eda を使用します
cover_type_counts = df_eda['Cover_Type'].value_counts().reset_index()
cover_type_counts.columns = ['Cover_Type', 'Count']
fig = px.bar(cover_type_counts, x='Cover_Type', y='Count',
            title="目的変数（Cover_Type）の分布",
            color='Cover_Type',
            labels={'Cover_Type': '森林被覆タイプ', 'Count': 'サンプル数'})
fig.show()


--- 1. 目的変数（Cover_Type）の分布 ---


In [10]:
# =============================================================================
# 2. 数値特徴量と目的変数の関係
# =============================================================================
print("\n--- 2. 数値特徴量と目的変数の関係 (インタラクティブな箱ひげ図) ---")
for col in numerical_features:
    fig = px.box(df_eda, x='Cover_Type', y=col, color='Cover_Type',
                    title=f"Cover_Typeごとの {col} の分布",
                     labels={'Cover_Type': '森林被覆タイプ'})
    fig.show()


--- 2. 数値特徴量と目的変数の関係 (インタラクティブな箱ひげ図) ---


In [11]:
# =============================================================================
# 3. カテゴリ特徴量と目的変数の関係
# =============================================================================
print("\n--- 3. カテゴリ特徴量と目的変数の関係 (インタラクティブな棒グラフ) ---")
# Wilderness_Area vs Cover_Type
fig = px.histogram(df_eda, x='Wilderness_Area', color='Cover_Type',
                    barmode='group',
                    title="Wilderness_AreaごとのCover_Type分布",
                    labels={'Wilderness_Area': '原生自然地域', 'Cover_Type': '森林被覆タイプ'})
fig.show()

# Soil_Type vs Cover_Type (種類が多いため、上位10種に絞って表示)
top_10_soil_types = df_eda['Soil_Type'].value_counts().nlargest(10).index
df_top_soil = df_eda[df_eda['Soil_Type'].isin(top_10_soil_types)]
    
fig = px.histogram(df_top_soil, x='Soil_Type', color='Cover_Type',
                    barmode='group',
                    title="Soil_TypeごとのCover_Type分布 (上位10種)",
                    category_orders={'Soil_Type': top_10_soil_types},
                    labels={'Soil_Type': '土壌タイプ', 'Cover_Type': '森林被覆タイプ'})
fig.update_xaxes(tickangle=45)
fig.show()


--- 3. カテゴリ特徴量と目的変数の関係 (インタラクティブな棒グラフ) ---


In [12]:
# =============================================================================
# 4. 数値特徴量間の相関
# =============================================================================
print("\n--- 4. 数値特徴量間の相関 (インタラクティブなヒートマップ) ---")
correlation_matrix = df_eda[numerical_features].corr()
fig = px.imshow(correlation_matrix,
                text_auto=True,
                aspect="auto",
                color_continuous_scale='RdBu_r', # 赤-白-青のカラースケール
                title="数値特徴量間の相関ヒートマップ")
fig.show()


--- 4. 数値特徴量間の相関 (インタラクティブなヒートマップ) ---


In [13]:
# ----------------------------------------------
# 特徴量作成(特徴量エンジニアリング)
# ----------------------------------------------
train_x["Euclidean_Distance_to_Hydrology"] = np.sqrt(train_x.Horizontal_Distance_To_Hydrology**2 + train_x.Vertical_Distance_To_Hydrology**2)
test_x["Euclidean_Distance_to_Hydrology"] = np.sqrt(test_x.Horizontal_Distance_To_Hydrology**2 + test_x.Vertical_Distance_To_Hydrology**2)
train_x["EVDtH"] = train_x.Elevation - train_x.Vertical_Distance_To_Hydrology
test_x["EVDtH"] = test_x.Elevation - test_x.Vertical_Distance_To_Hydrology
train_x["EHDtH"] = train_x.Elevation - train_x.Horizontal_Distance_To_Hydrology * 0.2
test_x["EHDtH"] = test_x.Elevation - test_x.Horizontal_Distance_To_Hydrology * 0.2
train_x["Hydro_Fire_1"] = train_x.Horizontal_Distance_To_Hydrology + train_x.Horizontal_Distance_To_Fire_Points
test_x["Hydro_Fire_1"] = test_x.Horizontal_Distance_To_Hydrology + test_x.Horizontal_Distance_To_Fire_Points
train_x["Hydro_Fire_2"] = abs(train_x.Horizontal_Distance_To_Hydrology - train_x.Horizontal_Distance_To_Fire_Points)
test_x["Hydro_Fire_2"] = abs(test_x.Horizontal_Distance_To_Hydrology - test_x.Horizontal_Distance_To_Fire_Points)
train_x["Hydro_Road_1"] = abs(train_x.Horizontal_Distance_To_Hydrology + train_x.Horizontal_Distance_To_Roadways)
test_x["Hydro_Road_1"] = abs(test_x.Horizontal_Distance_To_Hydrology + test_x.Horizontal_Distance_To_Roadways)
train_x["Hydro_Road_2"] = abs(train_x.Horizontal_Distance_To_Hydrology - train_x.Horizontal_Distance_To_Roadways)
test_x["Hydro_Road_2"] = abs(test_x.Horizontal_Distance_To_Hydrology - test_x.Horizontal_Distance_To_Roadways)
train_x["Fire_Road_1"] = abs(train_x.Horizontal_Distance_To_Fire_Points + train_x.Horizontal_Distance_To_Roadways)
test_x["Fire_Road_1"] = abs(test_x.Horizontal_Distance_To_Fire_Points + test_x.Horizontal_Distance_To_Roadways)
train_x["Fire_Road_2"] = abs(train_x.Horizontal_Distance_To_Fire_Points - train_x.Horizontal_Distance_To_Roadways)
test_x["Fire_Road_2"] = abs(test_x.Horizontal_Distance_To_Fire_Points - test_x.Horizontal_Distance_To_Roadways)
train_x["EHiElv"] = train_x.Horizontal_Distance_To_Roadways * train_x.Elevation
test_x["EHiElv"] = test_x.Horizontal_Distance_To_Roadways * test_x.Elevation
train_x["EViElv"] = train_x.Vertical_Distance_To_Hydrology * train_x.Elevation
test_x["EViElv"] = test_x.Vertical_Distance_To_Hydrology * test_x.Elevation
# train_x["Soil_Type12_32"] = train_x.Soil_Type32 + train_x.Soil_Type12
# test_x["Soil_Type12_32"] = test_x.Soil_Type32 + test_x.Soil_Type12
# train_x["Soil_Type23_22_32_33"] = train_x.Soil_Type23 + train_x.Soil_Type22 + train.Soil_Type32 + train_x.Soil_Type33
# test_x["Soil_Type23_22_32_33"] = test_x.Soil_Type23 + test_x.Soil_Type22 + test_x.Soil_Type32 + test_x.Soil_Type33
# train_x['binned_elevation'] = np.floor(train_x['Elevation'] / 50.0)
# test_x['binned_elevation'] = np.floor(test_x['Elevation'] / 50.0)
# train_x = train_x.drop([ "Soil_Type7", "Soil_Type15"], axis=1)
# test_x = test_x.drop(["Soil_Type7", "Soil_Type15"], axis=1)

# # 水源、発火点、道路までの水平距離をリストにまとめる
# dist_cols = ["Horizontal_Distance_To_Hydrology", 
#              "Horizontal_Distance_To_Fire_Points", 
#              "Horizontal_Distance_To_Roadways"]
# # 平均値と標準偏差を新しい特徴量として追加
# train_x["mean_dist"] = train_x[dist_cols].mean(axis=1)
# train_x["std_dist"] = train_x[dist_cols].std(axis=1)
# test_x["mean_dist"] = test_x[dist_cols].mean(axis=1)
# test_x["std_dist"] = test_x[dist_cols].std(axis=1)

# # 度をラジアンに変換してからsin, cosを計算
# rad = np.deg2rad(train_x["Aspect"])
# train_x["Aspect_sin"] = np.sin(rad)
# train_x["Aspect_cos"] = np.cos(rad)
# rad = np.deg2rad(test_x["Aspect"])
# test_x["Aspect_sin"] = np.sin(rad)
# test_x["Aspect_cos"] = np.cos(rad)

# # 水平距離が0の場合に備えて、微小な値(1e-6)を足して0除算を回避
# # 勾配 (Vertical / Horizontal)
# train_x["Hydrology_slope"] = train_x.Vertical_Distance_To_Hydrology / (train_x.Horizontal_Distance_To_Hydrology + 1e-6)
# test_x["Hydrology_slope"] = test_x.Vertical_Distance_To_Hydrology / (test_x.Horizontal_Distance_To_Hydrology + 1e-6)
# # 標高と何かの比率 (上で mean_dist を作成した場合)
# train_x["Elevation_over_mean_dist"] = train_x.Elevation / (train_x.mean_dist + 1e-6)
# test_x["Elevation_over_mean_dist"] = test_x.Elevation / (test_x.mean_dist + 1e-6)

train_x["Horizontal_Distance_To_Roadways_Log"] = np.log1p(train_x.Horizontal_Distance_To_Roadways)
test_x["Horizontal_Distance_To_Roadways_Log"] = np.log1p(test_x.Horizontal_Distance_To_Roadways)
train_x["Water Elevation"] = train_x.Elevation - train_x.Vertical_Distance_To_Hydrology
test_x["Water Elevation"] = test_x.Elevation - test_x.Vertical_Distance_To_Hydrology


In [14]:
# Mapping soil type to ELU code
ELU_CODE = {
    1:2702,2:2703,3:2704,4:2705,5:2706,6:2717,7:3501,8:3502,9:4201,
    10:4703,11:4704,12:4744,13:4758,14:5101,15:5151,16:6101,17:6102,
    18:6731,19:7101,20:7102,21:7103,22:7201,23:7202,24:7700,25:7701,
    26:7702,27:7709,28:7710,29:7745,30:7746,31:7755,32:7756,33:7757,
    34:7790,35:8703,36:8707,37:8708,38:8771,39:8772,40:8776
}

In [15]:
# Encode soil type ordinally
def categorical_encoding(input_df):
    data = input_df.copy()
    data['Soil_Type'] = 0
    for i in range(1,41):
        data['Soil_Type'] += i*data[f'Soil_Type{i}']
    return data

# Encode soil type
train_x = categorical_encoding(train_x)
test_x = categorical_encoding(test_x)

# Original soil features
soil_features = [f'Soil_Type{i}' for i in range(1,41)]

def climatic_zone(input_df):
    df = input_df.copy()
    df['Climatic_Zone'] = input_df['Soil_Type'].apply(
        lambda x: int(str(ELU_CODE[x])[0])
    )
    return df

# Climatic Zone
train_x = climatic_zone(train_x)
test_x = climatic_zone(test_x)

def geologic_zone(input_df):
    df = input_df.copy()
    df['Geologic_Zone'] = input_df['Soil_Type'].apply(
        lambda x: int(str(ELU_CODE[x])[1])
    )
    return df

# Geologic Zone
train_x = geologic_zone(train_x)
test_x = geologic_zone(test_x)

def surface_cover(input_df):
    # Group IDs
    no_desc = [7,8,14,15,16,17,19,20,21,23,35]
    stony = [6,12]
    very_stony = [2,9,18,26]
    extremely_stony = [1,22,24,25,27,28,29,30,31,32,33,34,36,37,38,39,40]
    rubbly = [3,4,5,10,11,13]

    # Create dictionary
    surface_cover = {i:0 for i in no_desc}
    surface_cover.update({i:1 for i in stony})
    surface_cover.update({i:2 for i in very_stony})
    surface_cover.update({i:3 for i in extremely_stony})
    surface_cover.update({i:4 for i in rubbly})
    
    # Create Feature
    df = input_df.copy()
    df['Surface_Cover'] = input_df['Soil_Type'].apply(
        lambda x: surface_cover[x]
    )
    return df

# Surface Cover
train_x = surface_cover(train_x)
test_x = surface_cover(test_x)

# def rock_size(input_df):
    
#     # Group IDs
#     no_desc = [7,8,14,15,16,17,19,20,21,23,35]
#     stones = [1,2,6,9,12,18,24,25,26,27,28,29,30,31,32,33,34,36,37,38,39,40]
#     boulders = [22]
#     rubble = [3,4,5,10,11,13]

#     # Create dictionary
#     rock_size = {i:0 for i in no_desc}
#     rock_size.update({i:1 for i in stones})
#     rock_size.update({i:2 for i in boulders})
#     rock_size.update({i:3 for i in rubble})
    
#     df = input_df.copy()
#     df['Rock_Size'] = input_df['Soil_Type'].apply(
#         lambda x: rock_size[x]
#     )
#     return df

# # Surface Cover
# train_x = rock_size(train_x)
# test_x = rock_size(test_x)

# def soiltype_interactions(data):
#     df = data.copy()
            
#     # Important Soil Types
#     df['Soil_12_32'] = df['Soil_Type32'] + df['Soil_Type12']
#     df['Soil_Type23_22_32_33'] = df['Soil_Type23'] + df['Soil_Type22'] + df['Soil_Type32'] + df['Soil_Type33']
    
#     # Soil Type Interactions
#     df['Soil29_Area1'] = df['Soil_Type29'] + df['Wilderness_Area1']
#     df['Soil3_Area4'] = df['Wilderness_Area4'] + df['Soil_Type3']
    
#     #  New Feature Interactions
#     df['Climate_Area2'] = df['Wilderness_Area2']*df['Climatic_Zone'] 
#     df['Climate_Area4'] = df['Wilderness_Area4']*df['Climatic_Zone'] 
#     df['Rock_Area1'] = df['Wilderness_Area1']*df['Rock_Size']    
#     df['Rock_Area3'] = df['Wilderness_Area3']*df['Rock_Size']  
#     df['Surface_Area1'] = df['Wilderness_Area1']*df['Surface_Cover'] 
#     df['Surface_Area2'] = df['Wilderness_Area2']*df['Surface_Cover']   
#     df['Surface_Area4'] = df['Wilderness_Area4']*df['Surface_Cover'] 
    
#     # Fill NA
#     df.fillna(0, inplace = True)
    
#     return df
    
# # Surface Cover
# train_x = soiltype_interactions(train_x)
# test_x = soiltype_interactions(test_x)

# Drop original soil features
train.drop(columns = soil_features, inplace = True)
test.drop(columns = soil_features, inplace = True)

In [16]:
# ------------------------------
# XGBoostの学習・推論・submit（ベースライン）
# ------------------------------
# 学習用にクラスラベルを修正（０〜６に対応させる）
# train_y = train_y - 1

model = XGBClassifier(objective="multi:softmax", num_class=7, random_state=rseed)
model.fit(train_x, train_y)
preds = model.predict(test_x)

print("\n--- 学習済みモデルの性能評価 ---")
train_preds = model.predict(train_x)
train_preds_proba = model.predict_proba(train_x)
accuracy = accuracy_score(train_y, train_preds)
logloss = log_loss(train_y, train_preds_proba)
print(f"訓練データに対する正解率 (Accuracy): {accuracy:.4f}")
print(f"訓練データに対するLogLoss: {logloss:.4f}")


# 提出用にクラスラベルを修正（１〜７に対応させる）
preds = preds + 1

submission = pd.DataFrame({
    "Id": test_id,
    "Cover_Type":  preds
})

submission.to_csv("submission.csv", index=False)


--- 学習済みモデルの性能評価 ---
訓練データに対する正解率 (Accuracy): 0.9972
訓練データに対するLogLoss: 0.0644
